## 1) 训练循环到底在做什么

一个标准的深度学习训练，本质是在反复执行：

>用当前参数 θ 计算损失 L(θ)，求梯度 g=∂L/∂θ，然后用优化器把 θ 往让 L 变小的方向更新。

训练循环就是把这件事工程化、可控化：可扩展（大数据/多卡）、可稳定（不炸梯度/不 NaN）、可复现（能重跑一致）、可观测（日志/指标）、可恢复（断点续训）。

### 1.1 最小训练循环

以 PyTorch 思维写（框架无关）：

In [ ]:
for epoch in range(E):
    model.train()
    for x, y in dataloader:
        y_hat = model(x)
        loss = loss_fn(y_hat, y)    #forward

        optimizer.zero_grad()   #清空上一轮累计的梯度
        loss.backward()         #backward：把梯度写到每个参数的.grad上
        optimizer.step()        #upgrad：用.grad更新参数

## 2) 训练循环的关键工程细节

### 2.1 梯度累积（gradient accumulation）

目的：显存不够，但你想要更大的等效 batch size。

做法：把多个 micro-batch 的梯度累加起来，最后再 step() 一次。

等效 batch：B_effective = B_micro × accum_steps × num_gpus

In [ ]:
optimizer.zero_grad()
for step, (x, y) in enumerate(dataloader):
    loss = loss_fn(model(x), y) / accum_steps
    loss.backward()

    if (step + 1) % accum_steps == 0:
        optimizer.step()
        optimizer.zero_grad()

### 2.2 梯度裁剪（gradient clipping）
Transformer 训练很常见：梯度偶发爆炸 → loss NaN。

典型做法：按范数裁剪（global norm）：

若 ‖g‖₂ > c，则令 g ← g * (c / ‖g‖₂)

这会保留方向、限制步长上限。常用 c=1.0。

### 2.3 学习率调度（scheduler）与 warmup
对 Transformer 来说，warmup 基本是标配：前期用小 lr，让 Adam 的二阶矩估计稳定，同时避免随机初始化阶段的大步更新把训练炸掉。

常见组合：

Warmup + Linear Decay（最经典，BERT/许多 finetune）

Warmup + Cosine Decay

预训练大模型：还会有更复杂的分段/循环策略，但核心仍是 warmup + 下降。

### 2.4 Mixed Precision（AMP）与 Loss Scaling
FP16/BF16 提速省显存，但 FP16 易下溢：梯度变 0。

解决：loss scaling：把 loss 乘一个 scale，反传后再把梯度除回去。

工程上用 AMP 自动做（含动态 scale），你需要能讲清楚“为什么 scale 能避免下溢”。

### 2.5 Dropout/评估模式切换（train/eval）
model.train()：dropout 生效（随机屏蔽），输出具有随机性

model.eval()：dropout 关闭，输出确定，用于验证/测试

验证要配 no_grad()：不建图，省显存省算力

### 2.6 断点续训（checkpoint）必须保存什么
只存模型参数不够，想“无缝恢复”，你至少要存：

model.state_dict()

optimizer.state_dict()（Adam 里有一堆动量/二阶矩状态）

scheduler.state_dict()（不然 lr 断档）

当前 step/epoch、随机种子状态（严格复现时）

**为什么 optimizer state 很重要？**
Adam 这类自适应方法每个参数都有 m, v，不恢复就等于换了优化器历史，会出现训练抖动甚至性能回退。

## 3) 优化器：从 SGD 到 AdamW，把“原理 + 直觉 + 适用场景”讲透
### 3.1 SGD（随机梯度下降）
更新：

>θ ← θ − η g

直觉：沿着当前 batch 的损失下降方向走一步。

缺点：噪声大、收敛慢、对条件数差的问题（不同方向曲率差异大）很敏感。

### 3.2 Momentum（动量）
核心：对梯度做指数滑动平均，让更新更“有惯性”。

一种常见写法：

>v ← μ v + g

>θ ← θ − η v

直觉：在一致方向上加速，抑制来回抖动（尤其在狭长谷底）。

Nesterov Momentum：先“看一眼未来位置”再算梯度（细节略），通常略更稳。

### 3.3 AdaGrad / RMSProp（自适应学习率的起点）
**AdaGrad：累积平方梯度**

>r ← r + g²

>θ ← θ − η * g / (sqrt(r)+ε)

问题：r 单调增，lr 越来越小，后期可能走不动。

**RMSProp：用滑动平均替代累积**

>r ← ρ r + (1−ρ) g²

>θ ← θ − η * g / (sqrt(r)+ε)

直觉：某个维度如果梯度长期大，就自动给它更小的步长。

## 3.4 Adam
Adam = Momentum（m，一阶矩） + RMSProp（v，二阶矩）

>m ← β1 m + (1−β1) g

>v ← β2 v + (1−β2) g²

因为初始化 m,v 从 0 开始，前期有偏差，所以要 bias correction：

>m̂ = m / (1−β1^t)

>v̂ = v / (1−β2^t)

最终更新：

>θ ← θ − η * m̂ / (sqrt(v̂)+ε)

**直觉：**

m̂ 给你“稳定方向”（降噪）

v̂ 给你“按维度缩放步长”（大梯度维度少走点）

bias correction 让前几步不至于被“初始化为 0”严重低估

## 4) AdamW：为什么 Transformer 里几乎都用它
### 4.1 L2 正则 vs Weight Decay 的差别
L2 正则：把 λ/2 * ||θ||² 加到 loss 里

梯度会变成：g_total = g + λθ

然后交给优化器（Adam）处理。

Weight Decay（权重衰减）：是“更新规则里直接衰减参数”

>θ ← (1 − ηλ) θ − η * update_from_grad

在 SGD 下这两者等价（因为更新是线性的）。

但在 Adam 下不等价：因为 Adam 会对梯度做自适应缩放，L2 正则项 λθ 也会被缩放，导致“不同参数被不一致地正则化”，这会影响泛化。

在 SGD 下这两者等价（因为更新是线性的）。

但在 Adam 下不等价：因为 Adam 会对梯度做自适应缩放，L2 正则项 λθ 也会被缩放，导致“不同参数被不一致地正则化”，这会影响泛化。

**AdamW 做的事：把 weight decay 从梯度里“解耦”出来，确保衰减行为不被自适应缩放扭曲。**

>面试一句话总结：AdamW = Adam + decoupled weight decay，解决 Adam 中 L2 正则被自适应缩放导致的非预期正则问题。

## 5) Transformer/NLP 训练的默认“靠谱配置”
### 5.1 微调（finetune）BERT/LLM encoder/decoder
optimizer：AdamW

lr：encoder 类常见 1e-5 ~ 5e-5；更大模型/更敏感任务往小

betas：(0.9, 0.999)

eps：1e-8

weight_decay：0.01（常见）

scheduler：warmup(比例 0.06~0.1) + linear decay

grad_clip：1.0

一般会对 bias 和 LayerNorm 权重 不做 weight decay（工程惯例）

### 从头预训练（pretrain）
仍常见 AdamW/Adafactor/LAMB 等（看规模与显存）

更大 batch + 更长 warmup

更强调吞吐（AMP、梯度累积、分布式）

## 手写 Adam / AdamW 更新公式
设第 𝑡 次更新时，参数为 𝜃_𝑡，当前 mini-batch 的梯度为
$$g_{t}=\nabla_{\theta} L\left(\theta_{t}\right)$$

超参：学习率$\alpha, \beta_{1}, \beta_{2} \in(0,1), \epsilon>0 .$

**Adam**
一阶矩（动量）
$$m_{t}=\beta_{1} m_{t-1}+\left(1-\beta_{1}\right) g_{t}$$

二阶矩（平方梯度的滑动平均）
$$v_{t}=\beta_{2} v_{t-1}+\left(1-\beta_{2}\right) g_{t}^{2}$$

这里$g_{t}^{2}$是按元素平方

Bias correction（关键）
$$\hat{m}_{t}=\frac{m_{t}}{1-\beta_{1}^{t}}, \quad \hat{v}_{t}=\frac{v_{t}}{1-\beta_{2}^{t}}$$

参数更新
$$\theta_{t+1}=\theta_{t}-\alpha \frac{\hat{m}_{t}}{\sqrt{\hat{v}_{t}}+\epsilon}$$

AdamW（Decoupled Weight Decay 单独写一项）

设 weight decay 系数 λ。

AdamW 的核心是：把权重衰减从梯度里拿出来，不参与 Adam 的自适应缩放。

因此更新写成“两部分相加”最清楚：

**(A) 先做 decoupled weight decay（衰减项）**
$$\theta_{t}^{\prime}=\theta_{t}-\alpha \lambda \theta_{t}$$

**(B) 再做 Adam 的自适应梯度更新（用同样的 𝑚,𝑣与 bias correction）**
$$\begin{array}{c}m_{t}=\beta_{1} m_{t-1}+\left(1-\beta_{1}\right) g_{t}, \quad v_{t}=\beta_{2} v_{t-1}+\left(1-\beta_{2}\right) g_{t}^{2} \\\hat{m}_{t}=\frac{m_{t}}{1-\beta_{1}^{t}}, \quad \hat{v}_{t}=\frac{v_{t}}{1-\beta_{2}^{t}} \\\theta_{t+1}=\theta_{t}^{\prime}-\alpha \frac{\hat{m}_{t}}{\sqrt{\hat{v}_{t}}+\epsilon}\end{array}$$

注意：这里的 𝜆𝜃_𝑡是“直接加到更新里”，不是加到梯度里让 Adam 缩放。

写一个你自己的训练循环代码，必须包含： 

grad accumulation 

AMP（含 scaler/unscale） 

grad clipping 

scheduler 

eval 切换与 no_grad 

checkpoint 保存 optimizer/scheduler 状态

In [ ]:
import os
import torch
from torch.nn.utils import clip_grad_norm_

def save_checkpoint(ckpt_path, model, optimizer, scheduler, scaler, epoch, global_step, best_metric):
    state = {
        "model": model.state_dict(),    #保存模型参数（权重）
        "optimizer": optimizer.state_dict(),    #保存优化器状态（Adamw的m/v等）
        "scheduler": scheduler.state_dict() if scheduler else None, #保存学习率调度状态（避免lr断档）
        "scaler": scaler.state_dict() if scaler else None,  #保存AMP的scaler状态（动态lossing scaler）
        "epoch": epoch,
        "global_step": global_step, #保存全局step（记录训练迭代）
        "best_metric": best_metric, #保存最优指标（用于early step / best model）
        "rng_state_cpu": torch.get_rng_state(), #保存cpu的随机数状态
        "rng_state_cuda": torch.cuda.get_rng_state_all() if torch.cuda.is_available else None, #保存cuda的随机数状态
    }
    os.makedirs(os.path.dirname(ckpt_path), exist_ok=True)  #确保checkpoint路径存在
    torch.save(state, ckpt_path)    #将state序列化到磁盘

def load_checkpoint(ckpt_path, model, optimizer, scheduler, scaler, map_location="cpu"):    
    state = torch.load(ckpt_path, map_location=map_location)
    model.load_state_dict(state["model"])   #恢复模型权重
    optimizer.load_state_dict(state["optimizer"])   #恢复优化器内部状态（m/v，step等）
    if scheduler and state.get("scheduler") is not None:
        scheduler.load_state_dict(state["scheduler"])   #恢复scheduler状态（当前lr，last_epoch等）
    if scaler and state.get("scaler") is not None:  #如果启用AMP，且ckpt中保存了scaler
        scaler.load_state_dict(state["scaler"]) #恢复scaler状态（动态scale的值）
    if state.get("rng_state_cpu") is not None:
        torch.set_rng_state(state["rng_state_cpu"]) #恢复rng cpu状态（复现实验用）
    if torch.cuda.is_available() and state.get("rng_state_cuda") is not None:
        torch.cuda.set_rng_state_all(state["rng_state_cuda"])   #恢复所有GPU的rng状态
    start_epoch = state.get("epoch", 0) #从ckpt读取初始的epoch，默认是0
    global_step = state.get("global_step", 0)   #从ckpt读取全局的step，默认是0
    best_metric = state.get("best_metric", None)    #从ckpt读取最优指标，可为空
    return start_epoch, global_step, best_metric

@torch.no_grad()
def evaluate(model, val_loader, device):
    model.eval()
    total_loss = 0.0
    total_count = 0
    for batch in val_loader:
        batch = move_batch_to_device(batch, device) #将batch移动到GPU/CPU
        loss = compute_loss(model, batch)   #计算验证loss
        bs = get_batch_size(batch)  #获取batch size
        total_loss += loss.item() * bs  #按样本加权累加（避免最后一批大小不同导致偏差）
        total_count += bs   #累加样本数
    avg_loss = total_loss / max(total_count, 1)
    metric = -avg_loss  # 示例：把指标设为 -loss（你可以换成 F1/EM/Acc）
    model.train()
    return {"val_loss": avg_loss, "metric": metric}

def trian(model, train_loader, val_loader, optimizer, scheduler, device, num_epochs, )


In [ ]:
import os 
from dataclasses import dataclass
from typing import Dict, Any, Optional

import torch
from torch import nn
from torch.utils.data import DataLoader

#配置：把训练循环里需要的超参与路径集中管理
@dataclass
class TrainConfig:
    device: str = "cuda" if torch.cuda.is_available else "cpu"  #训练设备
    epochs: int = 3 #训练伦数
    grad_accum_steps: int = 4   #梯度累积步数（micro-batch 累到这一步才 step）
    max_grad_norm: float = 1.0  #全局范数裁剪阈值（Transformer 常用 1.0）
    log_every: int = 50 #打印/记录频率（按 micro-step）
    eval_every_updates: int = 200   #每多少个“optimizer更新步”做一次验证
    ckpt_dir: str = "./crpt"    #checkpoint目录
    ckpt_name: str = "latest.pt"    #checkpoint文件名
    use_amp: bool = True    #是否开启AMP（fp16/bf16）
    amp_dtype: str = "fp16" #"fp16" or "bf16"（bf16 通常不需要 scaler）

#工具：保存 checkpoint（必须保存 optimizer/scheduler 状态）
def save_checkpoint(
        path: str,
        model: nn.Module,
        optimizer: torch.optim.Optimizer,
        scheduler: Optional[torch.optim.lr_scheduler._LRScheduler],
        scaler: Optional[torch.cuda.amp.GradScaler],
        extra: Dict[str, Any],
) -> None:
    os.makedirs(os.path.dirname(path), exist_ok=True)   #确保目录存在
    ckpt = {
        "model": model.state_dict(),    #保存模型参数
        "optimizer": optimizer.state_dict(),    #保存优化器状态（AdamW 的 m/v 等都在这里）
        "scheduler": scheduler.state_dict() if scheduler is not None else None, #保存学习率调度器状态
        "scaler": scaler.state_dict() if scaler is not None else None,  #保存 AMP scaler 状态（fp16 动态缩放需要）
        "extra": extra, #保存你自定义的训练进度/指标
        "rng_cpu": torch.get_rng_state(),   #保存 CPU 随机数状态（可复现）
        "rng_cuda": torch.cuda.get_rng_state_all() if torch.cuda.is_available() else None,  #保存cuda随机数状态
    }
    torch.save(ckpt, path)

#工具：加载 checkpoint（断点续训“无缝”恢复）
def load_checkpoint(
        path: str,
        model: nn.Moudle,
        optimizer: torch.optim.Optimizer,
        scheduler: Optional[torch.optim.lr_scheduler._LRscheduler],
        scaler: Optional[torch.cuda.amp.GradScaler],
        map_location: str = "cpu",
) -> Dict[str, Any]:
    ckpt = torch.load(path, map_location=map_location)  #读取checkpoint
    model.load_state_dict(ckpt["model"]) #恢复模型参数
    optimizer.load_state_dict(ckpt["optimizer"])    #恢复优化器状态（非常关键）
    if scheduler and ckpt.get("scheduler") is None:
        scheduler.load_state_dict(ckpt["scheduler"])    #恢复 scheduler（避免 lr 断档）
    if scaler and ckpt.get("scaler") is None:
        scaler.load_state_dict(ckpt["scaler"])  #恢复 scaler（避免动态缩放跳变）
    
    # 恢复 RNG（需要严格复现时很重要；不要求也可以不做）
    if ckpt.get("rng_cpu") is not None:
        torch.set_rng_state(ckpt["rng_cpu"])
    if ckpt.get("rng_cuda") is not None:
        torch.cuda.set_rng_state_all(ckpt["rng_cuda"])

    return ckpt.get("extra", {})    #返回你保存的“训练进度”等信息

#工具：验证（必须 eval() + no_grad）
@torch.no_grad()
def evaluate(model: nn.Module, val_loader: DataLoader, device: str) -> Dict[str, float]:
    model.eval()
    total_loss = 0.0
    total_tokens = 0    #可选：按 token 归一化（NLP 常用）

    loss_fn = nn.CrossEntropyLoss(ignore_index=-100)    #示例：LM/seq2seq 常用 ignore_index

    for batch in val_loader:
        #假设 batch 里有 input_ids, labels
        input_ids = batch["input_ids"].to(device)
        labels = batch["labels"].to(device)

        logits = model(input_ids)
        #logits: [B, T, V]，labels: [B, T] -> reshape 计算 token-level CE
        loss = loss_fn(logits.view(-1, logits.size(-1)), labels.view(-1))

        #统计：这里简单按 batch 累加；你也可以按 tokens 计
        total_loss += loss
        total_tokens += labels.numel()  #token数（可选）

    model.train()                                                                 # 评估结束切回 train（防止后续训练忘了）
    return {
        "val_loss": total_loss / max(1, len(val_loader)),                         # 平均验证 loss
    }

#训练循环：包含 grad accumulation + AMP(scaler/unscale) + grad clipping + scheduler + eval/no_grad + checkpoint
def train_loop(
        model: nn.Module,
        train_loader: DataLoader,
        val_loader: Optional[DataLoader],
        optimizer: torch.optim.Optimizer,
        scheduler: Optional[torch.optim.lr_scheduler._LRScheduler],
        cfg: TrainConfig,
        resume: bool = True
) -> None:
    device = cfg.device
    model.to(device)

    #AMP autocast 的 dtype：fp16 或 bf16（bf16 一般无需 GradScaler，但为了统一接口也可以不用 scaler）
    use_fp16 = cfg.use_amp and cfg.amp_dtype.lower() == "fp16" and device.startswith("cuda")
    use_bf16 = cfg.use_amp and cfg.amp_dtype.lower() == "bf16" and device.startswith("cuda")

    #GradScaler：仅 fp16 场景必需（bf16 通常不需要 loss scaling）
    scaler = torch.cuda.amp.GradScaler(enabled="fp16")  #fp16开启动态缩放，否则禁用

    #断点续训：加载模型/优化器/scheduler/scaler 以及 global_step 等训练进度
    ckpt_path = os.path.join(cfg.ckpt_dir, cfg.ckpt_name)   #checkpoint路径
    extra = {"epoch": 0, "update_step": 0, "micro_step": 0, "best_metric": 1e-9}    #初始化训练进度
    if resume and os.path.exists(ckpt_path):    #如果需要resume且存在ckpt
        extra = load_checkpoint(                #恢复所有状态
            ckpt_path, model, optimizer, scheduler, scaler, map_location="cpu"
        )

    #损失函数：示例为 token-level CE；分类任务替换即可
    loss_fn = nn.CrossEntropyLoss(ignore_index=-100)    #忽略padding的token

    update_step =  int(extra.get("update_step", 0))    #optimizer 更新步（每 accum 完才 +1）
    micro_step = int(extra.get("micro_step", 0))    #micro-batch步（每个batch+1）
    best_metric = int(extra.get("best_metric", 1e-9))   #记录最好指标（如最小 val_loss）

    model.train()
    optimizer.zero_grad(set_to_none=True)   #清空梯度（set_to_none 更省内存）

    for epoch in range(int(extra.get("epoch", 0)), cfg.epochs): #从恢复的epoch开始继续
        for batch in train_loader:
            micro_step += 1 #micro-step +1（每个 batch）

            input_ids = batch["input_ids"].to(device)
            labels = batch["labels"].to(device)

            # 2) AMP autocast：在 fp16/bf16 下用低精度计算 forward，提速省显存
            #    注意：autocast 只影响 forward/部分算子；backward 仍由 scaler 管
            autocast_enabled = (use_fp16 or use_bf16)   #是否使用autocast
            autocast_dtype = torch.float16 if use_fp16 else (torch.bfloat16 if use_bf16 else None) 

            with torch.autocast(device_type="cuda", dtype=autocast_dtype, enabled=autocast_enabled):
                logits = model(input_ids)   # forward：得到 logits
                loss = loss_fn(logits.view(-1, logits.size(-1)), labels.view(-1))   #计算 loss（token-level）
                loss = loss / cfg.grad_accum_steps  #关键：梯度累积时要除以 accum_steps

            #3) backward：fp16 用 scaler.scale(loss).backward() 避免梯度下溢；非 fp16 直接 backward
            if use_fp16:
                scaler.sacle(loss).backward()   #反传（loss 被放大，梯度随之放大
            else:
                loss.backward() #反传（bf16/fp32 直接做）
            
            #4) 梯度累积边界：凑够 grad_accum_steps 才做一次参数更新
            if micro_step % cfg.grad_accum_steps == 0:
                #4.1) unscale：把梯度从“放大后的尺度”还原回真实尺度，才能正确裁剪/检测 NaN/Inf
                if use_fp16:
                    scaler.unscale_(optimizer)  #让optimizer里的参数grad便会真实值
                
                #4.2) grad clipping：全局范数裁剪，防止偶发梯度爆炸（Transformer 常用）
                grad_norm = torch.nn.utils.clip_grad_norm_(
                    model.parameters(), cfg.max_grad_norm   
                )   #返回裁剪前（或后）的范数，便于日志

                #4.3) optimizer step：fp16 用 scaler.step；否则普通 step
                if use_fp16:
                    scaler.step(optimizer)  #若检测到 inf/nan，内部会跳过 step
                    scaler.update()     #更新动态scale（更稳）
                else:
                    optimizer.step()

                #4.4) scheduler step：通常 Transformer 以“每个 update_step”更新 lr
                if scheduler is not None:
                    scheduler.step()    #lr 调度（warmup/decay 在这里发生）

                #5) 日志：建议记录 update_step 对齐的指标（loss、lr、grad_norm）
                if update_step % max(1, cfg.eval_every_updates // 10) == 0:
                    lr = optimizer.param_groups[0][lr]  #获取当前 lr（示例取第一个 param group）
                    print(f"[epoch={epoch} upd={update_step}] lr={lr:.3e} "
                          f"loss={(loss.item()*cfg.grad_accum_steps):.4f} grad_norm={float(grad_norm):.3f}")
                    
                #6)eval：必须 model.eval() + no_grad（evaluate 里已经做了）
                if val_loader is not None and (update_step % cfg.eval_every_updates == 0):
                    metrics = evaluate(model, val_loader, device)   #评估（内部 no_grad + eval）
                    val_loss = metrics["val_loss"]

                    #7) checkpoint：保存 latest（并可按 best 保存一份）
                    extra = {                                                     # 把训练进度写进 extra
                        "epoch": epoch,
                        "update_step": update_step,
                        "micro_step": micro_step,
                        "best_metric": best_metric,
                        "last_val_loss": val_loss,
                    }
                    save_checkpoint(ckpt_path, model, optimizer, scheduler, scaler, extra)  #保存包含 optimizer/scheduler/scaler 的 ckpt

                    #8) best model（示例：val_loss 越小越好）
                    if val_loss < best_metric or best_metric < 0:
                        best_metric = val_loss
                        best_path = os.path.join(cfg.ckpt_dir, "best.pt")   #best extra 路径
                        extra["best_metric"] = best_metric                        # 写回 extra
                        save_checkpoint(best_path, model, optimizer, scheduler, scaler, extra)  # 保存 best

        # 每个 epoch 结束也建议存一次（防止 val_loader 为空时完全不存）
        extra = {
            "epoch": epoch + 1,                                                   # 下次从下一轮开始
            "update_step": update_step,
            "micro_step": micro_step,
            "best_metric": best_metric,
        }
        save_checkpoint(ckpt_path, model, optimizer, scheduler, scaler, extra)    # epoch end checkpoint

# -----------------------------
# 用法提示（你在自己工程里替换 model/loader/optimizer/scheduler 即可）
# -----------------------------
"""
你需要自己提供：
- model：forward(input_ids) -> logits
- train_loader / val_loader：batch 至少包含 "input_ids" 和 "labels"
- optimizer：torch.optim.AdamW
- scheduler：例如 transformers 里的 get_linear_schedule_with_warmup 或自定义 lr_scheduler

关键点已全部覆盖：
✅ grad accumulation: loss/accum_steps + micro_step%accum==0 才 step
✅ AMP: autocast + GradScaler(scale/backward, unscale_, step, update)
✅ grad clipping: clip_grad_norm_
✅ scheduler: 每次 update_step 后 scheduler.step()
✅ eval 切换与 no_grad: evaluate() 内 model.eval() + @torch.no_grad()
✅ checkpoint: 保存/加载 model+optimizer+scheduler(+scaler) + step/epoch
"""


AttributeError: module 'torch.nn' has no attribute 'Moudle'